In [1]:
# !python3 -m deeppavlov install squad_bert

### Imports and constants

In [1]:
import os
from torch import cuda, device

BASE_DIR = "./../../../"
DATA_DIR = os.path.join(BASE_DIR, "data/rucos")
SOURCES_DIR = os.path.join(BASE_DIR, "nti_ai_pipeline")
TRAIN_FILE_PATH = os.path.join(DATA_DIR, "rucos_train.jsonl")
VAL_FILE_PATH = os.path.join(DATA_DIR, "rucos_val.jsonl")
TEST_FILE_PATH = os.path.join(DATA_DIR, "rucos_test.jsonl")

BATCH_SIZE = 2
DEVICE = device("cuda" if cuda.is_available() else "cpu")
TRANSFORMER_NAME = "DeepPavlov/rubert-base-cased"
MOCK = True # turn off if run real pipeline, turn on if testing that things work

In [2]:
import sys
import importlib
import torch

sys.path.append(SOURCES_DIR)

import nti_rucos
import pipeline
from torch.utils import data as torch_data
from nti_rucos import modeling as rucos_modeling
from pipeline.saving import local_saver



my_modules = [nti_rucos, pipeline, rucos_modeling, local_saver]

for module in my_modules:
    importlib.reload(module)

### Loading data

In [4]:
container = nti_rucos.data.RucosContainer(TRAIN_FILE_PATH, VAL_FILE_PATH)

test_size = 0.001 if MOCK else 0.2
train_data, test_data = container.train_test_split(shuffle=True, test_size=test_size)

train_dataset = nti_rucos.data.RucosTrainDataset(train_data)
val_dataset = nti_rucos.data.RucosValDataset(test_data)

train_loader = torch_data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_loader = torch_data.DataLoader(val_dataset, batch_size=BATCH_SIZE)

### Defining model and trainer

In [3]:
proc = nti_rucos.modeling.RucosProcessor(TRANSFORMER_NAME, use_ner=False)
if MOCK:   
    model = rucos_modeling.mock_model.MockModel( # don't using SentPairBinaryClassifier because my laptop can't handle it with 12G of RAM
        use_ner=False
    )
else:
    model = rucos_modeling.SentPairBinaryClassifier(
        TRANSFORMER_NAME,
        droprate=0.2, 
        use_ner=False
    )
saver = pipeline.saving.local_saver.LocalSaver(save_dir='./saved_models')

manager = pipeline.modeling.ModelManager(model, proc, DEVICE, saver)

In [6]:
validator = nti_rucos.evaluating.RucosValidator()
# eval_val = validator.eval(manager, val_loader)

In [7]:
weights_updater = pipeline.WeightsUpdater(lr=7e-6, warmup=200, lr_end=1.7e-6, accum_iters=6, weight_decay=0.04, optimizer_class=torch.optim.AdamW) # NOTE: had set to 0 from 1e-2
trainer = pipeline.training.Trainer(validator, weights_updater)

### Running fit

In [8]:
# del model, manager
import gc, torch
gc.collect()
torch.cuda.empty_cache()

if MOCK:
    steps_betw_evals = 50
    max_step = 90
else:
    steps_betw_evals = 5000
    max_step = 50001
    
trainer.fit(train_loader, val_loader, manager, max_step=max_step, steps_betw_evals=steps_betw_evals, stop_patience=3)


_eval. Eval_value: 0.13698630034923553
Saved model. Eval value: 0.13698630034923553 Name: 584ccd79-a6bb-40b1-a3a9-2104302f32f2


### Pseudo labeling

In [4]:
pseudo_labeler = nti_rucos.sources.pseudo_labeling.RegressionPseudoLabeler(chosen_proportion=0.15, 
                                                                            pos_to_neg_proportion=0.5)

In [5]:
test_container = nti_rucos.data.RucosContainer(path=os.path.join(DATA_DIR, 'rucos_test.jsonl'), has_labels=False,
                                                  query_placeholder_union_mode="concatenate",
                                                  )
test_dataset = nti_rucos.data.RucosSubmDataset(test_container.get_data(), switch_texts=True)
if MOCK:
    test_dataset = torch_data.Subset(test_dataset, list(range(1000)))
    
test_loader = torch_data.DataLoader(test_dataset, batch_size=2)

In [6]:
features_labels = pseudo_labeler.run(manager, test_loader)

In [7]:
features_labels[:2]

[RucosSample(features=RucosSampleFeatures(text1='Как и год назад, 17 февраля 2008 года, когда на улицы Приштины вышли тысячи ликующих людей, власти Косово и на этот раз не пожалели денег на празднование независимости. Однако праздник для многих имеет горький привкус. 17 февраля исполнился год провозглашения независимости Косово. Примечательно, что именно с Косово начался развал Югославской Федерации. 20 лет назад, в связи с 600-летием исторической битвы на Косовом поле, проигранной сербами туркам, тогдашний сербский лидер Слободан Милошевич произнес речь, подогревшую антиалбанские настроения. Более того, год спустя он упразднил автономный статус Косово и таким образом придал импульс сепаратистским движениям в югославских республиках.\n@highlight\nБелград обвиняет ЕС в нарушении договоренностей по Косово\n@highlight\nЭксперт: Дробление Косово навредит самой Сербии\n@highlight\nРоссия противодействует работе миссии Евросоюза в Косово', text2='Слободан Милошевич Благодаря этому компромисс

### Submitting

In [4]:
# best_manager = trainer.load_best_manager()
# best_manager = managing_model.ModelManager.load(saver, '9ab8822d-1d50-4d8d-904b-2e7a70dca78f')
best_manager = manager

submitter = nti_rucos.rucos_submitting.RucosSubmitter(subm_dir=os.path.join(BASE_DIR, 'submissions'))

# test loader is taken from Pseudo labeling step
submitter.create_submission(
    best_manager, test_loader, subm_file_name="subm_single_bert_50000_steps.jsonl",
    )